In [1]:
import httpx
from pysolr import Solr

CREATE CORE

In [2]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'authorities',
        'configSet': '_default',
    },
}

response = httpx.post('http://localhost:8983/api/cores', headers=headers, json=json_data)
response

<Response [200 OK]>

In [6]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'subjects',
        'configSet': '_default',
    },
}

response = httpx.post('http://localhost:8983/api/cores', headers=headers, json=json_data, timeout=20)
response

<Response [200 OK]>

In [2]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'acervo',
        'configSet': '_default',
    },
}

response = httpx.post('http://localhost:8983/api/cores', headers=headers, json=json_data, timeout=20)
response

<Response [200 OK]>

In [3]:
solr = Solr('http://localhost:8983/solr/acervo/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":23,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-18"}},\n  "status":"OK"}\n'

In [3]:
from rdflib import Graph

In [4]:
g = Graph()
token = "n80002329"
rdf = f'https://id.loc.gov/authorities/names/{token}.madsrdf.rdf'
g.parse(rdf)  

<Graph identifier=N6456db544e6b4fc6a5191c115dc43b2a (<class 'rdflib.graph.Graph'>)>

In [12]:
g.serialize("machado.ttl")

<Graph identifier=N2d15a036cefb44b1a0ee7ad1835773e8 (<class 'rdflib.graph.Graph'>)>

In [50]:
q = """
PREFIX lc: <http://id.loc.gov/authorities/names/>
PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  ?variantLabel
WHERE { lc:"""+token+" madsrdf:hasVariant ?o . \
?o madsrdf:variantLabel ?variantLabel . }"

r = g.query(q)
for i in r:
    print(i.variantLabel.value)

Assis, Joaquim Maria Machado de, 1839-1908
Assis, Machado de, 1839-1908
De Assis, Joaquim Maria Machado, 1839-1908
De Assis, Machado, 1839-1908
Machado de Assis, Joaquim Maria, 1839-1908
Machado de Assis, Joaquín María, 1839-1908
Mashado de Assiz, Zhoakin, 1839-1908


In [5]:
def DocName(g, token):

    doc = {"id": token, "type": "PersonalName" }

    qName = """
PREFIX lc: <http://id.loc.gov/authorities/names/>
PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#> 
SELECT  ?o
WHERE { lc:"""+token+" madsrdf:authoritativeLabel ?o}"

    r = g.query(qName)

    for i in r:
        doc['name'] = i.o.value

    qFullerName = """
PREFIX lc: <http://id.loc.gov/authorities/names/>
PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  ?fullerName
WHERE { lc:"""+token+" madsrdf:fullerName ?o . \
?o rdfs:label ?fullerName . }"

    r = g.query(qFullerName)

    for i in r:
        doc['fullerName'] = i.fullerName.value
    
    qVariantes = """
PREFIX lc: <http://id.loc.gov/authorities/names/>
PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  ?variantLabel
WHERE { lc:"""+token+" madsrdf:hasVariant ?o . \
?o madsrdf:variantLabel ?variantLabel . }"

    r = g.query(qVariantes)

    variants = list()
    for i in r:
        variants.append(i.variantLabel.value)
    doc["variants"] = variants
    
    return doc

doc = DocName(g, token)   
doc


{'id': 'n80002329',
 'type': 'PersonalName',
 'name': 'Machado de Assis, 1839-1908',
 'fullerName': 'Joaquim Maria Machado',
 'variants': ['Assis, Joaquim Maria Machado de, 1839-1908',
  'Assis, Machado de, 1839-1908',
  'De Assis, Joaquim Maria Machado, 1839-1908',
  'De Assis, Machado, 1839-1908',
  'Machado de Assis, Joaquim Maria, 1839-1908',
  'Machado de Assis, Joaquín María, 1839-1908',
  'Mashado de Assiz, Zhoakin, 1839-1908']}

In [9]:
solr.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":1418}}\n'

In [3]:
solr.delete(q='*:*')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">329</int>\n</lst>\n</response>\n'

In [5]:
work = {
        "id": "wk-bk-23-4",
        "hasInstance": {"add": [f"in-bk-23-5"]}, 
    }

solr.add([work], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":110}}\n'